# レッスン4: オーディオエフェクトとダイナミクス

前回まで学んだこと：
- 基本的な音響合成（発振器、エンベロープ）
- フィルターによる音色デザイン
- 楽器音の作成

今回のレッスンで学ぶこと：
- **リバーブ（残響）**で音に空間的な広がりを与える
- **ディレイ（遅延）**でエコー効果を作る
- **コーラス**で音を豊かにする
- **ダイナミクス**（音量制御）の基本

🎵 ゴール：プロっぽい音響効果を理解し、音楽に深みを与える技術を身につける

## 準備：ライブラリのセットアップ

In [ ]:
# Google Colab環境の確認とセットアップ
import sys

# Colab環境かどうかを確認
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 Google Colab環境を設定中...")
    
    # 必要なパッケージをインストール
    !pip install numpy scipy matplotlib ipython
    
    # GitHubからライブラリをクローン
    !git clone https://github.com/ggszk/simple-audio-programming.git
    
    # パスを追加
    sys.path.append('/content/simple-audio-programming')
    
    print("✅ セットアップ完了！")
    print("📝 このノートブックを自分用にコピーするには:")
    print("   ファイル → ドライブにコピーを保存")
    
else:
    print("🏠 ローカル環境で実行中")
    print("📝 audio_libがインストールされていることを確認してください")

# インポート文（メインモジュールから直接インポート）
from audio_lib import (
    AudioConfig, SineWave, ADSREnvelope, Reverb, Distortion,
    note_to_frequency, save_audio
)
from audio_lib.synthesis.envelopes import apply_envelope
from audio_lib.effects.audio_effects import Delay, Chorus, Compressor, apply_compression
from IPython.display import Audio, display
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

config = AudioConfig()

print("\n🎵 オーディオエフェクトと音響処理の学習を始めましょう！")

## 4.1 リバーブ（残響）：音に空間を与える

リバーブは部屋や空間での音の反響をシミュレートする効果です。

- **コンサートホール**：長い残響
- **小さな部屋**：短い残響
- **屋外**：ほぼ残響なし

音楽では空間的な広がりや深みを演出するために使用されます。

In [ ]:
# シンプルなピアノ音を作成（前回の復習）
def create_simple_piano(note_freq, duration=2.0):
    """シンプルなピアノ音"""
    sine = SineWave(frequency=note_freq)
    samples = sine.generate(config, duration)
    
    # ピアノ風エンベロープ
    envelope = ADSREnvelope(
        attack_time=0.01,
        decay_time=0.3,
        sustain_level=0.4,
        release_time=1.5
    )
    
    return apply_envelope(samples, envelope, config, duration)

# ドライ音（リバーブなし）
dry_piano = create_simple_piano(note_to_frequency('C4'))

print("ドライ音（リバーブなし）:")
display(Audio(dry_piano, rate=config.sample_rate))

# リバーブありの音
reverb = Reverb(room_size=0.8, damping=0.5, wet_level=0.3)
wet_piano = reverb.apply(dry_piano, config.sample_rate)

print("リバーブあり（大きな部屋）:")
display(Audio(wet_piano, rate=config.sample_rate))

### 🎧 聞き比べてみよう
- ドライ音：直接的で近い感じ
- リバーブあり：空間的で遠い感じ、音の尻尾が長い

### リバーブパラメータの効果

In [ ]:
# 異なるリバーブ設定を比較
base_sound = create_simple_piano(note_to_frequency('A4'), 1.5)

# 設定1：小さな部屋
reverb_small = Reverb(room_size=0.3, damping=0.7, wet_level=0.2)
small_room = reverb_small.apply(base_sound, config.sample_rate)

print("小さな部屋（room_size=0.3, damping=0.7）:")
display(Audio(small_room, rate=config.sample_rate))

# 設定2：大きなホール
reverb_hall = Reverb(room_size=0.9, damping=0.2, wet_level=0.4)
concert_hall = reverb_hall.apply(base_sound, config.sample_rate)

print("コンサートホール（room_size=0.9, damping=0.2）:")
display(Audio(concert_hall, rate=config.sample_rate))

# 設定3：極端な設定
reverb_extreme = Reverb(room_size=0.95, damping=0.1, wet_level=0.6)
extreme_reverb = reverb_extreme.apply(base_sound, config.sample_rate)

print("極端なリバーブ（room_size=0.95, damping=0.1）:")
display(Audio(extreme_reverb, rate=config.sample_rate))

## 4.2 ディレイ（遅延）：エコー効果を作る

ディレイは音を一定時間遅らせて元の音に重ねる効果です。

- **短いディレイ**：音の厚みや広がり
- **長いディレイ**：明確なエコー効果
- **フィードバック**：エコーの繰り返し

In [ ]:
# ディレイ効果を試してみよう
base_sound = create_simple_piano(note_to_frequency('E4'), 1.0)

print("元の音:")
display(Audio(base_sound, rate=config.sample_rate))

# 短いディレイ（音の厚み）
delay_short = Delay(delay_time=0.1, feedback=0.3, wet_level=0.4)
short_delay_sound = delay_short.apply(base_sound, config.sample_rate)

print("短いディレイ（0.1秒）:")
display(Audio(short_delay_sound, rate=config.sample_rate))

# 長いディレイ（明確なエコー）
delay_long = Delay(delay_time=0.5, feedback=0.4, wet_level=0.5)
long_delay_sound = delay_long.apply(base_sound, config.sample_rate)

print("長いディレイ（0.5秒）:")
display(Audio(long_delay_sound, rate=config.sample_rate))

# フィードバック多め（エコーの繰り返し）
delay_feedback = Delay(delay_time=0.3, feedback=0.6, wet_level=0.4)
feedback_sound = delay_feedback.apply(base_sound, config.sample_rate)

print("フィードバック多め（エコーの繰り返し）:")
display(Audio(feedback_sound, rate=config.sample_rate))

## 4.3 コーラス：音を豊かにする

コーラスは音を少しずつピッチや時間をずらしてコピーし、重ね合わせる効果です。
まるで複数の楽器や声で演奏しているような豊かさを生み出します。

In [ ]:
# コーラス効果を試してみよう
base_sound = create_simple_piano(note_to_frequency('G4'), 2.0)

print("元の音:")
display(Audio(base_sound, rate=config.sample_rate))

# 軽いコーラス
chorus_light = Chorus(rate=2.0, depth=0.3, wet_level=0.4)
light_chorus_sound = chorus_light.apply(base_sound, config.sample_rate)

print("軽いコーラス:")
display(Audio(light_chorus_sound, rate=config.sample_rate))

# 深いコーラス
chorus_deep = Chorus(rate=1.5, depth=0.6, wet_level=0.6)
deep_chorus_sound = chorus_deep.apply(base_sound, config.sample_rate)

print("深いコーラス:")
display(Audio(deep_chorus_sound, rate=config.sample_rate))

# 速いコーラス（ビブラート的）
chorus_fast = Chorus(rate=5.0, depth=0.4, wet_level=0.5)
fast_chorus_sound = chorus_fast.apply(base_sound, config.sample_rate)

print("速いコーラス（ビブラート的）:")
display(Audio(fast_chorus_sound, rate=config.sample_rate))

## 4.4 エフェクトの組み合わせ：プロフェッショナルな音作り

In [ ]:
def create_lush_piano(note_freq, duration=3.0):
    """豊かなエフェクトをかけたピアノ音"""
    
    # 基本のピアノ音
    base_sound = create_simple_piano(note_freq, duration)
    
    # エフェクトチェーン：コーラス → ディレイ → リバーブ
    
    # 1. コーラスで豊かさを追加
    chorus = Chorus(rate=2.5, depth=0.4, wet_level=0.3)
    sound_with_chorus = chorus.apply(base_sound, config.sample_rate)
    
    # 2. 軽いディレイで空間感を追加
    delay = Delay(delay_time=0.15, feedback=0.25, wet_level=0.2)
    sound_with_delay = delay.apply(sound_with_chorus, config.sample_rate)
    
    # 3. リバーブで最終的な空間を作る
    reverb = Reverb(room_size=0.7, damping=0.4, wet_level=0.3)
    final_sound = reverb.apply(sound_with_delay, config.sample_rate)
    
    return final_sound

# 比較してみよう
dry_sound = create_simple_piano(note_to_frequency('C4'), 3.0)
lush_sound = create_lush_piano(note_to_frequency('C4'), 3.0)

print("ドライなピアノ音:")
display(Audio(dry_sound, rate=config.sample_rate))

print("豊かなエフェクト付きピアノ音:")
display(Audio(lush_sound, rate=config.sample_rate))

## 4.5 ダイナミクス処理の基本

ダイナミクスは音量の変化を制御する技術です。

- **コンプレッサー**：音量差を小さくする
- **リミッター**：音量の上限を設定
- **ゲート**：小さな音をカット

In [ ]:
# シンプルなコンプレッサー効果を実装
def simple_compressor(samples, threshold=0.7, ratio=4.0):
    """シンプルなコンプレッサー
    
    Args:
        samples: 入力音声
        threshold: コンプレッションが始まるレベル
        ratio: 圧縮比（4.0なら4:1の圧縮）
    """
    compressed = samples.copy()
    
    # 各サンプルに対してコンプレッション適用
    for i in range(len(compressed)):
        sample_level = abs(compressed[i])
        
        if sample_level > threshold:
            # 閾値を超えた分を圧縮
            excess = sample_level - threshold
            compressed_excess = excess / ratio
            new_level = threshold + compressed_excess
            
            # 元の符号を保持
            if compressed[i] >= 0:
                compressed[i] = new_level
            else:
                compressed[i] = -new_level
    
    return compressed

# 動的な音量変化を持つ音を作成
def create_dynamic_sound(duration=4.0):
    """音量が変化する音を作成"""
    freq = note_to_frequency('A4')
    sine = SineWave(frequency=freq)
    samples = sine.generate(config, duration)
    
    # 時間とともに変化する音量エンベロープ
    time_samples = len(samples)
    volume_envelope = np.ones(time_samples)
    
    # 不規則な音量変化を作成
    for i in range(time_samples):
        t = i / time_samples
        # 複数の正弦波で複雑な音量変化
        volume = 0.5 + 0.3 * np.sin(t * 8 * np.pi) + 0.2 * np.sin(t * 20 * np.pi)
        volume_envelope[i] = max(0.1, min(1.0, volume))
    
    return samples * volume_envelope

# ダイナミクス処理の比較
dynamic_sound = create_dynamic_sound()
compressed_sound = simple_compressor(dynamic_sound, threshold=0.6, ratio=3.0)

print("元の音（音量変化あり）:")
display(Audio(dynamic_sound, rate=config.sample_rate))

print("コンプレッサー適用後（音量変化を抑制）:")
display(Audio(compressed_sound, rate=config.sample_rate))

# 波形の比較
plt.figure(figsize=(12, 6))
time = np.linspace(0, 4, len(dynamic_sound))

plt.subplot(2, 1, 1)
plt.plot(time, dynamic_sound, alpha=0.7)
plt.title('元の音（音量変化あり）')
plt.ylabel('振幅')
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(time, compressed_sound, alpha=0.7, color='orange')
plt.title('コンプレッサー適用後')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4.6 実践演習：音楽フレーズにエフェクトを適用

In [ ]:
# 簡単なメロディーを作成してエフェクトを比較
def create_melody_with_effects():
    """メロディーにさまざまなエフェクトを適用"""
    
    # メロディー：C-E-G-C（上のド）
    notes = ['C4', 'E4', 'G4', 'C5']
    note_duration = 1.0
    
    melody_samples = []
    for note in notes:
        freq = note_to_frequency(note)
        note_sound = create_simple_piano(freq, note_duration)
        melody_samples.append(note_sound)
    
    # メロディーを連結
    full_melody = np.concatenate(melody_samples)
    
    return full_melody

# ベースメロディー
base_melody = create_melody_with_effects()

print("1. 元のメロディー（エフェクトなし）:")
display(Audio(base_melody, rate=config.sample_rate))

# リバーブのみ
reverb = Reverb(room_size=0.8, damping=0.3, wet_level=0.4)
melody_reverb = reverb.apply(base_melody, config.sample_rate)

print("2. リバーブのみ:")
display(Audio(melody_reverb, rate=config.sample_rate))

# ディレイのみ
delay = Delay(delay_time=0.25, feedback=0.4, wet_level=0.4)
melody_delay = delay.apply(base_melody, config.sample_rate)

print("3. ディレイのみ:")
display(Audio(melody_delay, rate=config.sample_rate))

# コーラスのみ
chorus = Chorus(rate=3.0, depth=0.5, wet_level=0.5)
melody_chorus = chorus.apply(base_melody, config.sample_rate)

print("4. コーラスのみ:")
display(Audio(melody_chorus, rate=config.sample_rate))

# 全エフェクト組み合わせ
melody_with_chorus = chorus.apply(base_melody, config.sample_rate)
melody_with_delay = delay.apply(melody_with_chorus, config.sample_rate)
melody_full_fx = reverb.apply(melody_with_delay, config.sample_rate)

print("5. 全エフェクト組み合わせ（コーラス→ディレイ→リバーブ）:")
display(Audio(melody_full_fx, rate=config.sample_rate))

## 4.7 チャレンジ課題

### 🎯 課題1：自分だけのエフェクトプリセット
異なる音楽ジャンルに適したエフェクト設定を作ってみよう

In [ ]:
def create_genre_preset(base_sound, genre="pop"):
    """ジャンル別エフェクトプリセット"""
    
    if genre == "pop":
        # ポップス：適度なコーラスと短いリバーブ
        chorus = Chorus(rate=2.0, depth=0.3, wet_level=0.3)
        reverb = Reverb(room_size=0.5, damping=0.6, wet_level=0.2)
        
        sound = chorus.apply(base_sound, config.sample_rate)
        sound = reverb.apply(sound, config.sample_rate)
        
    elif genre == "ambient":
        # アンビエント：深いリバーブと長いディレイ
        delay = Delay(delay_time=0.6, feedback=0.5, wet_level=0.4)
        reverb = Reverb(room_size=0.9, damping=0.2, wet_level=0.6)
        chorus = Chorus(rate=1.0, depth=0.6, wet_level=0.4)
        
        sound = chorus.apply(base_sound, config.sample_rate)
        sound = delay.apply(sound, config.sample_rate)
        sound = reverb.apply(sound, config.sample_rate)
        
    elif genre == "dry":
        # ドライ：軽いコンプレッションのみ
        sound = simple_compressor(base_sound, threshold=0.8, ratio=2.0)
        
    else:  # デフォルト
        sound = base_sound
    
    return sound

# テスト用の音
test_sound = create_simple_piano(note_to_frequency('A4'), 2.0)

# 各ジャンルプリセットを試す
genres = ["dry", "pop", "ambient"]

for genre in genres:
    processed_sound = create_genre_preset(test_sound, genre)
    print(f"{genre.upper()}プリセット:")
    display(Audio(processed_sound, rate=config.sample_rate))
    print()

### 🎯 課題2：エフェクトの順序による違い
エフェクトを適用する順序で音がどう変わるか実験してみよう

In [ ]:
# エフェクトチェーンの順序比較
base_sound = create_simple_piano(note_to_frequency('C4'), 2.0)

# チェーン1：リバーブ → ディレイ
reverb = Reverb(room_size=0.7, damping=0.4, wet_level=0.4)
delay = Delay(delay_time=0.3, feedback=0.3, wet_level=0.4)

chain1 = reverb.apply(base_sound, config.sample_rate)
chain1 = delay.apply(chain1, config.sample_rate)

print("チェーン1：リバーブ → ディレイ")
display(Audio(chain1, rate=config.sample_rate))

# チェーン2：ディレイ → リバーブ
chain2 = delay.apply(base_sound, config.sample_rate)
chain2 = reverb.apply(chain2, config.sample_rate)

print("チェーン2：ディレイ → リバーブ")
display(Audio(chain2, rate=config.sample_rate))

print("\n🎧 違いを聞き比べてみよう！")
print("チェーン1：ディレイ音にもリバーブがかかる")
print("チェーン2：リバーブ音がディレイで繰り返される")

### 🎯 課題3：ライブパフォーマンス風のエフェクト切り替え

In [ ]:
# 時間軸でエフェクトが変化する音を作成
def create_performance_sound():
    """ライブパフォーマンス風：時間とともにエフェクトが変化"""
    
    # 4つのセクション、それぞれ2秒
    sections = []
    base_freq = note_to_frequency('G4')
    
    # セクション1：ドライ
    section1 = create_simple_piano(base_freq, 2.0)
    sections.append(section1)
    
    # セクション2：コーラス追加
    section2 = create_simple_piano(base_freq, 2.0)
    chorus = Chorus(rate=3.0, depth=0.4, wet_level=0.5)
    section2 = chorus.apply(section2, config.sample_rate)
    sections.append(section2)
    
    # セクション3：ディレイ追加
    section3 = create_simple_piano(base_freq, 2.0)
    section3 = chorus.apply(section3, config.sample_rate)
    delay = Delay(delay_time=0.25, feedback=0.4, wet_level=0.4)
    section3 = delay.apply(section3, config.sample_rate)
    sections.append(section3)
    
    # セクション4：フルエフェクト
    section4 = create_simple_piano(base_freq, 2.0)
    section4 = chorus.apply(section4, config.sample_rate)
    section4 = delay.apply(section4, config.sample_rate)
    reverb = Reverb(room_size=0.8, damping=0.3, wet_level=0.5)
    section4 = reverb.apply(section4, config.sample_rate)
    sections.append(section4)
    
    # 全セクションを連結
    return np.concatenate(sections)

performance_sound = create_performance_sound()

print("ライブパフォーマンス風エフェクト変化：")
print("0-2秒：ドライ → 2-4秒：コーラス → 4-6秒：ディレイ追加 → 6-8秒：フルエフェクト")
display(Audio(performance_sound, rate=config.sample_rate))

## まとめ

### 今回学んだこと：
1. **リバーブ**：空間的な広がりと深みを与える
2. **ディレイ**：エコー効果と音の厚みを作る
3. **コーラス**：音を豊かにし、複数楽器感を演出
4. **ダイナミクス処理**：音量変化の制御
5. **エフェクトチェーン**：複数エフェクトの組み合わせ
6. **ジャンル別プリセット**：用途に応じた設定

### 🎵 音楽制作のコツ：
- **エフェクトは適度に**：やりすぎは禁物
- **順序が重要**：エフェクトの適用順で音が大きく変わる
- **ジャンルに合わせる**：音楽スタイルに適した設定を選ぶ
- **コンテキストを考慮**：楽曲全体の中での役割を意識

### 次回予告：レッスン5
- **MIDI**システムと音楽データ
- **シーケンサー**プログラミング
- **自動作曲**の基礎
- **リズムパターン**の作成

### 🏠 宿題
1. 自分の好きな音楽ジャンル用のエフェクトプリセットを作ってみよう
2. 簡単なメロディーに3つ以上のエフェクトを組み合わせてみよう
3. エフェクトチェーンの順序を変えて、音の違いを確認してみよう